<a href="https://colab.research.google.com/github/WisamSaddique/AI-Study-Assistant/blob/main/final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install necessary libraries if not already installed
!pip install transformers torch sentence-transformers faiss-cpu numpy PyPDF2 python-docx gradio datasets

# Import necessary libraries
import nltk
import torch
from transformers import pipeline, AutoModelForQuestionAnswering, AutoTokenizer, Trainer, TrainingArguments
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import PyPDF2
import docx
import os
import gradio as gr
from nltk.tokenize import sent_tokenize
from datasets import load_dataset # Import datasets

# Download necessary NLTK data
nltk.download('punkt', quiet=True)
# Download the punkt_tab resource which is required by PunktTokenizer
nltk.download('punkt_tab', quiet=True)

class StudyAssistant:
    def __init__(self):
        """Initialize the Study Assistant with all required models"""
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        print(f"Using device: {self.device}")

        # Initialize document storage
        self.documents = {}
        self.document_embeddings = None
        self.index = None

        # Load models (with progress indicators)
        print("Loading summarization model...")
        self.summarizer = pipeline(
            "summarization",
            model="facebook/bart-large-cnn",
            device=0 if self.device == 'cuda' else -1
        )

        print("Loading question answering model...")
        self.qa_model = pipeline(
            "question-answering",
            model="deepset/roberta-base-squad2",
            device=0 if self.device == 'cuda' else -1
        )

        print("Loading embedding model for semantic search...")
        self.embedding_model = SentenceTransformer('all-MiniLM-L6-v2', device=self.device)

        print("Study Assistant initialized successfully!")

    def add_document(self, name, content):
        """Add a document to the assistant's knowledge base"""
        # Store the document
        self.documents[name] = content

        # Update embeddings for search
        self.update_embeddings()

        return f"Document '{name}' added successfully!"

    def update_embeddings(self):
        """Update embeddings for all documents"""
        # Create text chunks for embeddings
        all_chunks = []
        chunk_to_doc_map = []

        for doc_name, content in self.documents.items():
            # Split content into sentences
            sentences = sent_tokenize(content)

            # Create overlapping chunks of ~3-5 sentences for better context
            chunks = []
            for i in range(0, len(sentences), 3):
                chunk = " ".join(sentences[i:i+5])
                chunks.append(chunk)
                all_chunks.append(chunk)
                chunk_to_doc_map.append(doc_name)

        # Generate embeddings
        if all_chunks:
            embeddings = self.embedding_model.encode(all_chunks)

            # Store embeddings and mapping
            self.document_embeddings = embeddings
            self.chunk_to_doc_map = chunk_to_doc_map
            self.all_chunks = all_chunks # Store all_chunks as an instance variable

            # Create FAISS index for fast similarity search
            dimension = embeddings.shape[1]
            self.index = faiss.IndexFlatL2(dimension)
            self.index.add(np.array(embeddings).astype('float32'))

            print(f"Updated embeddings for {len(self.documents)} documents with {len(all_chunks)} chunks")

    def extract_text_from_pdf(self, pdf_file):
        """Extract text from a PDF file"""
        text = ""
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        for page in pdf_reader.pages:
            text += page.extract_text() + "\n"
        return text

    def extract_text_from_docx(self, docx_file):
        """Extract text from a DOCX file"""
        doc = docx.Document(docx_file)
        text = ""
        for para in doc.paragraphs:
            text += para.text + "\n"
        return text

    def process_uploaded_file(self, file):
        """Process an uploaded file and extract its content"""
        file_name = os.path.basename(file.name)
        file_ext = os.path.splitext(file_name)[1].lower()

        if file_ext == '.pdf':
            content = self.extract_text_from_pdf(file.name)
        elif file_ext == '.docx':
            content = self.extract_text_from_docx(file.name)
        elif file_ext == '.txt':
            with open(file.name, 'r', encoding='utf-8') as f:
                content = f.read()
        else:
            return f"Unsupported file type: {file_ext}"

        return self.add_document(file_name, content)

    def summarize(self, text, max_length=150, min_length=50):
        """Generate a summary of the provided text"""
        # For long texts, we need to split and summarize each part
        if len(text.split()) > 1000:
            # Split into chunks of ~1000 words
            chunks = []
            sentences = sent_tokenize(text)
            current_chunk = []
            current_length = 0

            for sentence in sentences:
                words = sentence.split()
                if current_length + len(words) <= 1000:
                    current_chunk.append(sentence)
                    current_length += len(words)
                else:
                    chunks.append(" ".join(current_chunk))
                    current_chunk = [sentence]
                    current_length = len(words)

            if current_chunk:
                chunks.append(" ".join(current_chunk))

            # Summarize each chunk
            summaries = []
            for chunk in chunks:
                summary = self.summarizer(chunk, max_length=max_length//len(chunks),
                                         min_length=min_length//len(chunks))[0]["summary_text"]
                summaries.append(summary)

            # Combine summaries
            return " ".join(summaries)
        else:
            # For shorter texts, summarize directly
            return self.summarizer(text, max_length=max_length, min_length=min_length)[0]["summary_text"]

    def summarize_document(self, doc_name, max_length=150, min_length=50):
        """Summarize a specific document"""
        if doc_name in self.documents:
            return self.summarize(self.documents[doc_name], max_length, min_length)
        else:
            return f"Document '{doc_name}' not found!"

    def find_relevant_context(self, query, top_k=3):
        """Find the most relevant context for a given query"""
        if not self.document_embeddings is not None:
            return "No documents available for search!"

        # Embed the query
        query_embedding = self.embedding_model.encode([query])

        # Search for the most similar chunks
        D, I = self.index.search(np.array(query_embedding).astype('float32'), top_k)

        # Collect relevant chunks and their source documents
        relevant_chunks = []
        source_docs = set()

        for idx in I[0]:
            # Ensure idx is within the bounds of self.chunk_to_doc_map and self.all_chunks
            if idx < len(self.chunk_to_doc_map) and idx < len(self.all_chunks):
                chunk = self.chunk_to_doc_map[idx]
                source_docs.add(chunk)
                relevant_chunks.append(self.all_chunks[idx])
            else:
                 print(f"Warning: Index {idx} out of bounds for chunk mapping or all_chunks.")

        # Join the relevant chunks to form the context
        context = " ".join(relevant_chunks)

        return context, list(source_docs)

    def answer_question(self, question):
        """Answer a question based on available documents"""
        if not self.documents:
            return "I don't have any study materials to answer from yet. Please add some documents first!"
        if not self.index:
            return "Document embeddings not created yet. Please add documents first."

        # Create query embedding
        query_embedding = self.embedding_model.encode([question])

        # Search for similar chunks
        top_k = 5
        D, I = self.index.search(np.array(query_embedding).astype('float32'), top_k)

        # Get relevant chunks and create context
        relevant_chunks = []
        relevant_docs = []
        for idx in I[0]:
            # Ensure idx is within the bounds of self.chunk_to_doc_map and self.all_chunks
             if idx < len(self.chunk_to_doc_map) and idx < len(self.all_chunks):
                relevant_chunks.append(self.all_chunks[idx])
                relevant_docs.append(self.chunk_to_doc_map[idx])
             else:
                 print(f"Warning: Index {idx} out of bounds for chunk mapping or all_chunks.")


        context = " ".join(relevant_chunks)
        unique_docs = list(set(relevant_docs))

        if not context:
             return "Could not find relevant context for your question in the documents."


        # Generate answer using QA model
        answer = self.qa_model(question=question, context=context)


        # Format the response
        response = f"Answer: {answer['answer']}\n\n"
        response += f"Confidence: {answer['score']:.2f}\n"
        response += f"Source documents: {', '.join(unique_docs)}"

        return response

# TRAINING WITH SQUAD DATASET

def train_qa_model():
    """
    Train a question answering model using the SQuAD dataset
    Note: This is a simplified training process for demonstration
    """

    print("Loading SQuAD dataset...")
    squad = load_dataset("squad")

    print("Preparing tokenizer...")
    model_name = "distilbert-base-uncased"  # Smaller model for Colab
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # Function to preprocess the dataset
    def preprocess_function(examples):
        questions = [q.strip() for q in examples["question"]]
        inputs = tokenizer(
            questions,
            examples["context"],
            max_length=384,
            truncation="only_second",
            stride=128,
            return_overflowing_tokens=True,
            return_offsets_mapping=True,
            padding="max_length",
        )

        offset_mapping = inputs.pop("offset_mapping")
        sample_map = inputs.pop("overflow_to_sample_mapping")
        answers = examples["answers"]
        start_positions = []
        end_positions = []

        for i, offset in enumerate(offset_mapping):
            sample_idx = sample_map[i]
            answer = answers[sample_idx]
            start_char = answer["answer_start"][0]
            end_char = answer["answer_start"][0] + len(answer["text"][0])
            sequence_ids = inputs.sequence_ids(i)

            # Find the start and end of the context
            idx = 0
            while sequence_ids[idx] != 1:
                idx += 1
            context_start = idx
            while sequence_ids[idx] == 1:
                idx += 1
            context_end = idx - 1

            # If the answer is not fully inside the context, label is (0, 0)
            if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
                start_positions.append(0)
                end_positions.append(0)
            else:
                # Otherwise, find the token corresponding to the start/end character
                idx = context_start
                while idx <= context_end and offset[idx][0] <= start_char:
                    idx += 1
                start_positions.append(idx - 1)

                idx = context_end
                while idx >= context_start and offset[idx][1] >= end_char:
                    idx -= 1
                end_positions.append(idx + 1)

        inputs["start_positions"] = start_positions
        inputs["end_positions"] = end_positions
        return inputs

    print("Preprocessing dataset...")
    tokenized_squad = squad.map(preprocess_function, batched=True, remove_columns=squad["train"].column_names)

    print("Initializing model...")
    model = AutoModelForQuestionAnswering.from_pretrained(model_name)

    # Set training arguments
    training_args = TrainingArguments(
        output_dir="./results",
        evaluation_strategy="epoch",
        learning_rate=3e-5,
        per_device_train_batch_size=8,  # Adjust based on available memory
        per_device_eval_batch_size=8,
        num_train_epochs=1,  # Just 1 epoch for demonstration
        weight_decay=0.01,
    )

    print("Starting training...")
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_squad["train"],
        eval_dataset=tokenized_squad["validation"],
        tokenizer=tokenizer,
    )

    trainer.train()

    print("Saving model...")
    model_path = "./trained_qa_model"
    model.save_pretrained(model_path)
    tokenizer.save_pretrained(model_path)

    print(f"Model trained and saved to {model_path}")
    return model_path

# UI IMPLEMENTATION

def create_gradio_interface(assistant):
    """Create a Gradio interface for the study assistant"""

    # File upload for documents
    def handle_file_upload(files):
        results = []
        for file in files:
            result = assistant.process_uploaded_file(file)
            results.append(result)
        return "\n".join(results)

    # Add text document
    def add_text_document(name, content):
        if not name:
            return "Please provide a document name!"
        return assistant.add_document(name, content)

    # Summarize document
    def summarize_doc(doc_name, max_length, min_length):
        return assistant.summarize_document(doc_name, max_length, min_length)

    # List available documents
    def list_documents():
        if not assistant.documents:
            return "No documents available."
        doc_list = "\n".join([f"- {name} ({len(content.split())} words)"
                             for name, content in assistant.documents.items()])
        return f"Available documents:\n{doc_list}"

    # Answer questions
    def answer_question(question):
        return assistant.answer_question(question)

    # Create the interface
    with gr.Blocks(title="AI Study Assistant") as demo:
        gr.Markdown("# AI Study Assistant")
        gr.Markdown("Upload your study materials and ask questions about them!")

        with gr.Tab("Add Documents"):
            with gr.Row():
                with gr.Column():
                    file_input = gr.File(file_count="multiple", label="Upload Files (PDF, DOCX, TXT)")
                    upload_button = gr.Button("Upload Files")
                    upload_output = gr.Textbox(label="Upload Status")
                    upload_button.click(handle_file_upload, inputs=[file_input], outputs=upload_output)

                with gr.Column():
                    doc_name = gr.Textbox(label="Document Name")
                    doc_content = gr.Textbox(label="Document Content", lines=10)
                    add_doc_button = gr.Button("Add Document")
                    doc_output = gr.Textbox(label="Status")
                    add_doc_button.click(add_text_document, inputs=[doc_name, doc_content], outputs=doc_output)

            list_docs_button = gr.Button("List Available Documents")
            docs_list = gr.Textbox(label="Available Documents", lines=5)
            list_docs_button.click(list_documents, inputs=[], outputs=docs_list)

        with gr.Tab("Summarize"):
            with gr.Row():
                with gr.Column():
                    sum_doc_name = gr.Textbox(label="Document Name")
                    max_length = gr.Slider(50, 500, 150, step=10, label="Maximum Summary Length")
                    min_length = gr.Slider(30, 200, 50, step=10, label="Minimum Summary Length")
                    summarize_button = gr.Button("Summarize Document")

                with gr.Column():
                    summary_output = gr.Textbox(label="Summary", lines=10)

            summarize_button.click(summarize_doc, inputs=[sum_doc_name, max_length, min_length], outputs=summary_output)

        with gr.Tab("Ask Questions"):
            question_input = gr.Textbox(label="Your Question")
            ask_button = gr.Button("Ask")
            answer_output = gr.Textbox(label="Answer", lines=10)

            ask_button.click(answer_question, inputs=[question_input], outputs=answer_output)

        gr.Markdown("### Instructions")
        gr.Markdown("""
        1. Start by uploading your study materials (PDF, DOCX, or text)
        2. Use the Summarize tab to get concise summaries of your documents
        3. Ask questions about your materials in the Questions tab
        """)

    return demo

# MAIN EXECUTION
def main():
    """Main function to run the study assistant"""
    print("Initializing AI Study Assistant...")
    assistant = StudyAssistant()

    # Uncomment to train a custom QA model (requires significant time and resources)
    # train_qa_model()

    # Create and launch the Gradio interface
    demo = create_gradio_interface(assistant)

    # Add some sample documents for testing
    assistant.add_document("Sample Notes", """
    Neural networks are a series of algorithms that mimic the operations of a human brain to recognize relationships between vast amounts of data.
    They are used in a variety of applications in computer vision, natural language processing, and playing board games.

    Convolutional Neural Networks (CNNs) are particularly effective for image recognition tasks.
    CNNs use convolutional layers that apply filters to detect features like edges, textures, and more complex patterns.

    Recurrent Neural Networks (RNNs) are designed for sequential data like text or time series.
    They have connections that form directed cycles, allowing the network to maintain information about previous inputs.
    Long Short-Term Memory (LSTM) networks are a special kind of RNN that can learn long-term dependencies.
    """)

    print("Launching Gradio interface...")
    demo.launch(share=True)  # Set share=True to create a public link

if __name__ == "__main__":
    main()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Study Assistant initialized successfully!
Updated embeddings for 1 documents with 3 chunks
Launching Gradio interface...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a9e9d9706a158899a0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
